# EP Spiral Transfer Trajectory Viewer

Select a thruster (auto-detected from `trajectory_*.csv`) and visualize the trajectory animation.

In [1]:
from pathlib import Path
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

import ui_plotter  # expects ui_plotter.py in the same folder (or on PYTHONPATH)

DATA_DIR = Path(".")  # change if your CSVs are in a subfolder

In [2]:
def detect_trajectories(data_dir: Path):
    """
    Finds trajectory_*.csv files and returns a dict:
      label -> filepath
    label is derived from filename: trajectory_LPH.csv -> LPH
    """
    files = sorted(data_dir.glob("trajectory_*.csv"))
    mapping = {}
    for f in files:
        label = f.stem.replace("trajectory_", "")  # e.g., "LPH"
        mapping[label] = str(f)
    return mapping

TRAJ = detect_trajectories(DATA_DIR)

print("Notebook working dir:", Path(".").resolve())
print("Detected CSVs:")
for k, v in TRAJ.items():
    print(f"  {k}: {v}")

if not TRAJ:
    raise FileNotFoundError(f"No trajectory_*.csv files found in {DATA_DIR.resolve()}")


Notebook working dir: /Users/aparnakrishnan/miniforge3/envs/aste404/miniproject/EP_Spiral_Transfers
Detected CSVs:
  HPH: trajectory_HPH.csv
  HPI: trajectory_HPI.csv
  LPH: trajectory_LPH.csv
  LPI: trajectory_LPI.csv
  mass_test: trajectory_mass_test.csv


In [4]:
# --- widgets ---
thruster_dropdown = widgets.Dropdown(
    options=sorted(TRAJ.keys()),
    value=sorted(TRAJ.keys())[0],
    description="Thruster:",
    layout=widgets.Layout(width="320px")
)

thin_slider = widgets.IntSlider(
    value=5000, min=1, max=50000, step=500,
    description="thin_step:",
    continuous_update=False,
    layout=widgets.Layout(width="520px")
)

delay_slider = widgets.FloatSlider(
    value=1.5, min=0.0, max=5.0, step=0.1,
    description="delay (s):",
    continuous_update=False,
    layout=widgets.Layout(width="520px")
)

interval_slider = widgets.IntSlider(
    value=20, min=5, max=100, step=5,
    description="interval ms:",
    continuous_update=False,
    layout=widgets.Layout(width="520px")
)

repeat_checkbox = widgets.Checkbox(
    value=False,
    description="repeat animation",
)

run_btn = widgets.Button(description="Visualize", button_style="success")
out = widgets.Output()

def on_run_clicked(_):
    with out:
        clear_output(wait=True)
        label = thruster_dropdown.value
        csv_path = TRAJ[label]
        print(f"Selected: {label}")
        print(f"CSV: {csv_path}")

        # Run animation
        ui_plotter.animate_from_csv(
            csv_path=csv_path,
            thin_step=int(thin_slider.value),
            interval_ms=int(interval_slider.value),
            repeat=bool(repeat_checkbox.value),
            show=True
        )

run_btn.on_click(on_run_clicked)

# --- layout ---
controls = widgets.VBox([
    thruster_dropdown,
    thin_slider,
    delay_slider,
    interval_slider,
    repeat_checkbox,
    run_btn
])

display(controls, out)


Output()

In [5]:
refresh_btn = widgets.Button(description="Refresh CSV list", button_style="info")

def on_refresh(_):
    global TRAJ
    TRAJ = detect_trajectories(DATA_DIR)
    thruster_dropdown.options = sorted(TRAJ.keys())
    if TRAJ:
        thruster_dropdown.value = sorted(TRAJ.keys())[0]
    with out:
        clear_output(wait=True)
        print("Refreshed. Detected:")
        for k, v in TRAJ.items():
            print(f"  {k}: {v}")

refresh_btn.on_click(on_refresh)
display(refresh_btn)


Button(button_style='info', description='Refresh CSV list', style=ButtonStyle())